In [ ]:
import pandas as pd

# Import and read data
filename = 'nebraska'
df = pd.read_csv(f"{filename}.csv")

# Clean phone numbers
nophone = df[df['Phone'].isna()]
df = df[df['Phone'].notna()]
df['Phone'] = df['Phone'].astype(str).str.replace(r'\.0$', '', regex=True).str.replace('^1', '', regex=True)
df['check'] = df['Phone'].apply(lambda x: len(x)!= 10)
long_numbers = df[df['check']]
df = df[~df['check']].drop('check', axis=1)
df['Phone'] = df['Phone'].apply(lambda x: f"{x[:3]}-{x[3:6]}-{x[6:]}")

# Set account type and plan type
df['Account Type'] = 'Rep Account'
df['PlanType']  = "AVS Dental"

# Clean address and state
stateless = df[df['State'].isna()]
listofstates= ['NE','Nebraska']
df = df[df['State'].isin(listofstates)]
df['State'] = df['State'].fillna('')
df['Address2'] = df['Address2'].fillna(df['Address1'].str.split(r'Suite\s',expand=True)[1])

# Clean notes
df['Notes'] = df['Notes'].fillna('')
df['Notes'] = df['Notes'].apply(lambda x: f" -- FROM Rep's PhoneBurner Account {x}")

# Set territory and campaign name
def territory(df):
    state = df['State']
    if state in ["Missouri", "MO"]:
        return 'Missouri'
    elif state in ["Kentucky", "KY"]:
        return "Kentucky"
    elif state in ["Oregon", "OR"]:
        return "Oregon"
    elif state in ["Tennessee", "TN"]:
        return "Tennessee"
    elif state in ["Massachusetts", "MA"]:
        return "Massachusetts"
    elif state in ["New Jersey", "NJ"]:
        return "New Jersey"
    elif state == "RI":
        return "Rhode Island"
    elif state in ["Connecticut", "CT"]:
        return "Connecticut"
    elif state in ["Maine", "ME"]:
        return "Maine"
    elif state in ["NH", "New Hampshire"]:
        return "New Hampshire"
    elif state in ["CA", "California"]:
        return "Anaheim"
    elif state in ["TX", "Texas"]:
        return "TX - DALLAS"
    elif state in ["AZ", "Arizona"]:
        return "Arizona"
    elif state in ["NV", "Nevada"]:
        return "Nevada"
    elif state in ["NY", "New York"]:
        return "New York"
    elif state in ["Illinois", "IL"]:
        return "Illinois"
    elif state in ["Colorado", "CO"]:
        return "Colorado"
    elif state in ["Indiana", "IN"]:
        return "Indiana"
    elif state in ["Maryland", "MD"]:
        return "Maryland"
    elif state in ["PA", "Pennsylvania"]:
        return "Pennsylvania"
    elif state in ["VA", "Virginia"]:
        return "Virginia"
    elif state in ["UT", "Utah"]:
        return "Utah"
    elif state in ["Ohio", "OH"]:
        return "Ohio"
    else:
        return "Outside State"

df['Territory'] = df.apply(territory, axis=1)
df['Campaign Name'] = df['Territory'].str.upper() + ' - 

# Create new column with formatted address
df['Full Address'] = df['Address1'] + ', ' + df['City'] + ', ' + df['State'] + ', ' + df['Zipcode'].astype(str)

# Clean phone numbers
df['Phone'] = df['Phone'].str.replace(r'\D', '', regex=True)
df['Phone'] = df['Phone'].apply(lambda x: f"+1{x}" if len(x) == 10 else x)

# Drop unnecessary columns
df = df.drop(['Address1', 'Address2', 'City', 'State', 'Zipcode'], axis=1)

# Reorder columns
df = df[['Campaign Name', 'Full Name', 'Phone', 'Territory', 'Notes', 'Full Address']]

# Add timestamp to filename and save as CSV
timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
filename = f"phoneburner_data_{timestamp}.csv"
df.to_csv(filename, index=False)

print(f"Saved cleaned data to {filename}")
